### 10 yr mean

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import esmlab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy
import cartopy.crs as ccrs
import seawater as sw
from seawater.library import T90conv
from scipy import stats
import pop_tools 

from pint import UnitRegistry
import utils
import ncar_jobqueue
from dask.distributed import Client

units = UnitRegistry()
units.define('micromoles_per_kilogram = umol / kg') # used in WOA datasets

Cannot write to data cache folder '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


In [3]:
# cluster = ncar_jobqueue.NCARCluster(project='ncgd0011')
# cluster.adapt(minimum_jobs=0, maximum_jobs=30)
# client = Client(cluster)
# client

### MONTHLY

#### Define CESM case and variable

In [12]:
case = 'cesm22cocco.G1850ECOIAF.T62_g17.006'
user = 'kristenk'
path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/month_1'
#path = '/glade/campaign/cgd/oce/projects/besome/cesm-cases/' + case + '/ocn/proc/tseries/month_1'
variables = ['FG_ALT_CO2']

#### Get tseries files

In [13]:
%%time
ds_list = []
print(f'reading {case}')
ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.{v}.031101-033012.nc'))       
  
    
    
    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True, chunks={'z_t' : 1})

    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading cesm22cocco.G1850ECOIAF.T62_g17.006
FG_ALT_CO2
CPU times: user 91.2 ms, sys: 27.4 ms, total: 119 ms
Wall time: 283 ms


In [14]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [15]:
%%time 
cesm_ann_ds=ds.isel(time=slice(120,240)).mean(dim='time').compute()

CPU times: user 683 ms, sys: 154 ms, total: 836 ms
Wall time: 843 ms


In [17]:
for v in variables:
    outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean_last10yr/mean_last10yr_'+v+'.nc'
    cesm_ann_ds[v].to_netcdf(outfile)

### ANNUAL

In [8]:
case = 'cesm22cocco.G1850ECOIAF.T62_g17.003'
#path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/year_1'
path = '/glade/campaign/cgd/oce/projects/besome/cesm-cases/' + case + '/ocn/proc/tseries/year_1'
variables = ['CaCO3_FLUX_IN','CaCO3_REMIN'] 

In [9]:
%%time

ds_list = []

print(f'reading {case}')

ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.ecosys.nyear1.{v}.0249-0310.nc'))

    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True)

    
    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading cesm22cocco.G1850ECOIAF.T62_g17.003
CaCO3_FLUX_IN
CaCO3_REMIN
CPU times: user 843 ms, sys: 121 ms, total: 964 ms
Wall time: 6.39 s


In [10]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','z_t_150m','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [11]:
ds=ds.mean(dim='time')

In [12]:
for v in variables:
        outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
        ds[v].to_netcdf(outfile)